In [1]:
import sys
sys.path.append("..")

In [2]:
from TimeGAN.Model import Model

In [3]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
from matplotlib import pyplot as plt

In [4]:
model = Model(seq_len=24, 
              hidden_dim=34, 
              noise_dim=32, 
              dim=128, 
              batch_size=128, 
              learning_rate=5e-4,
              data_ret_dir="../Data/ReturnDataDAX.csv", 
              data_abs_dir="../Data/PriceDataDAX.csv")

In [5]:
model.load_model(model_dir="../TimeGAN/SavedPKLs/synthesizer_DAX_returns_Hidden_dim34_500.pkl")

In [6]:
data = model.rescaled_cum_return(model.generate_synthetic(synthetic_size=100000))
DataTimeGAN = pd.DataFrame(data=data)

Synthetic data generation: 100%|█████████████████████████████████████████████████████| 782/782 [03:09<00:00,  4.07it/s]


In [11]:
T = 24/250
r = 0.01

In [12]:
class MonteCarloPricer():    
    def _price_european(self, St, r, df):
        self.St = St
        self.r = r
        dfST = df.iloc[:,-1] * self.St
        
        if self.Type == "Call":
            Payouts = dfST[dfST > self.K] - self.K
        elif self.Type == "Put":
            Payouts = self.K - dfST[dfST < self.K]
        
        x = Payouts.sum()/len(dfST)      
        return self.__discount(x)   
    
    def _price_digital(self,St, r, df, Q, q):
        self.St = St
        self.r = r
        self.Q = Q
        self.q = q
        dfST = df.iloc[:,-1] * self.St
        
        if self.PayoutType == "Cash":
            if self.Type == "Call":
                Payouts = len(dfST[dfST > self.K]) * self.Q
            elif self.Type == "Put":
                Payouts = len(dfST[dfST < self.K]) * self.Q
            x = Payouts/len(dfST)
            return self.__discount(x)
            
        elif self.PayoutType == "Asset":
            if self.Type == "Call":
                Payouts = dfST[dfST > self.K]
            elif self.Type == "Put":
                Payouts = dfST[dfST < self.K]
            x = Payouts.sum()/len(dfST)
            return self.__discount(x)
    
    def _price_asian(self, St, r, df, AverageType, AveragePeriod):
        self.St = St
        self.r = r
        self.AverageType = AverageType
        self.AveragePeriod = AveragePeriod
        
        meanDf = df.iloc[:, df.shape[1]-self.AveragePeriod:df.shape[1]]*self.St
        
        if self.AverageType == "Geometric":
            meanDf = scipy.mean(meanDf,axis=1)    
        elif self.AverageType == "Arithmetic":
            meanDf = scipy.stats.gmean(meanDf,axis=1)
        
        if self.PayoutType == "AveragePrice":
            if self.Type == "Call":
                Payouts = meanDf[meanDf > self.K] - self.K
            elif self.Type == "Put":
                Payouts = self.K - meanDf[self.K > meanDf]
        elif self.PayoutType == "AverageStrike": 
            dfST = df.iloc[:,-1] * self.St
            if self.Type == "Call":
                Payouts = dfST - meanDf
            elif self.Type == "Put":
                Payouts = meanDf - dfST
            Payouts = Payouts[Payouts>0]
        
        x = Payouts.sum()/meanDf.shape[0]
        return self.__discount(x)
    
    def _price_chooser(self, St, r, df):
        self.St = St
        self.r = r
        dfSt1 = df[round((df.shape[1]/self.T2)*self.T1,0)] * self.St
        dfST2 = df.iloc[:,-1] * self.St
        
        i = 0
        Payouts = []
        while i < len(df):
            if dfSt1[i] > self.K:
                Payouts.append(max(dfST2[i]-self.K,0))
            else:
                Payouts.append(max(self.K-dfST2[i],0))
            i += 1
        x = pd.DataFrame(Payouts).sum()/dfST2.shape[0]
        return self.__discount(x)
    
    def _price_floating_strike_lookback(self, St, r, df):
        self.St = St
        self.r = r
        dfST = df.iloc[:,-1] * self.St
        
        if self.Type == "Call":
            dfMin = df.min(axis=1) * self.St
            Payouts = dfST - dfMin
            
        elif self.Type == "Put":
            dfMax = df.max(axis=1) * self.St
            Payouts = dfMax - dfST
        
        x = Payouts.sum()/len(dfST)
        return self.__discount(x)        
    
    def __discount(self, x):
        return float(x * np.exp(-self.r*self.T))

In [13]:
class Option(MonteCarloPricer):
    def __init__(self, T, K):
        self.T = T
        self.K = K
        
    def _calculate_d1_d2(self, St, r, sigma, q):
        self.St = St
        self.r = r
        self.sigma = sigma
        self.q = q
        
        self.d1 = (np.log(self.St/self.K) + (self.r-self.q + self.sigma**2/2)*self.T)/(self.sigma*np.sqrt(self.T))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T)

In [14]:
class EuropeanCall(Option):
    def __init__(self, T, K):
        super().__init__(T, K)
        self.Type = "Call"
    
    def price_using_BS(self, St, r, sigma, q=0):
        self._calculate_d1_d2(St, r, sigma, q)
        b = r - q
        return self.St * np.exp((b-self.r)*self.T) * norm.cdf(self.d1, 0, 1) - self.K * np.exp(-self.r*self.T) * norm.cdf(self.d2, 0, 1)
    
    def price_using_monte_carlo(self,St, r, df):  
        return self._price_european(St , r, df)       

In [15]:
class EuropeanPut(Option):
    def __init__(self, T, K):
        super().__init__(T, K)
        self.Type = "Put"    

    def price_using_BS(self, St, r, sigma, q=0):
        self._calculate_d1_d2(St, r, sigma, q)
        b = r - q
        return self.K * np.exp(-self.r*self.T) * norm.cdf(-self.d2, 0, 1) - self.St * np.exp((b-self.r)*self.T) * norm.cdf(-self.d1, 0, 1) 
    
    def price_using_monte_carlo(self,St, r, df):    
        return self._price_european(St , r, df)   

In [16]:
class DigitalOption(Option):
    def __init__(self, T, K, PayoutType):
        super().__init__(T, K)
        self.PayoutType = PayoutType
    
    def price_closed_form(self, St, r, sigma, Q, q):
        self._calculate_d1_d2(St ,r , sigma, q)
        self.Q = Q
    
    def price_using_monte_carlo(self, St, r, df, Q=0, q=0):    
        return self._price_digital(St, r, df, Q, q)  

In [17]:
class DigitalCall(DigitalOption):
    def __init__(self, T, K, PayoutType):
        super().__init__(T, K, PayoutType)
        self.Type = "Call"
    
    def price_closed_form(self, St, r, sigma, Q=0, q=0):
        super().price_closed_form(St, r, sigma, Q, q)
        if self.PayoutType == "Cash":
            return self.Q * np.exp(-self.r*self.T) * norm.cdf(self.d2, 0, 1)
        elif self.PayoutType == "Asset":
            return self.St * np.exp(-self.q*self.T) * norm.cdf(self.d1, 0, 1)

In [18]:
class DigitalPut(DigitalOption):
    def __init__(self, T, K, PayoutType):
        super().__init__(T, K, PayoutType)
        self.Type = "Put"
    
    def price_closed_form(self, St, r, sigma, Q=0, q=0):
        super().price_closed_form(St, r, sigma, Q, q)
        if self.PayoutType == "Cash":
            return self.Q * np.exp(-self.r*self.T) * norm.cdf(-self.d2, 0, 1)
        elif self.PayoutType == "Asset":
            return self.St * np.exp(-self.q*self.T) * norm.cdf(-self.d1, 0, 1)

In [19]:
class AsianOption(Option):
    def __init__(self, T, PayoutType, K):
        self.PayoutType = PayoutType
        if self.PayoutType == "AveragePrice":
            super().__init__(T ,K)
        elif self.PayoutType == "AverageStrike":
            self.T = T
    
    def price_closed_form(self, St, r, sigma, q):
        if self.PayoutType == "AveragePrice":
            self.St = St
            self.r = r
            self.sigma = sigma/np.sqrt(3)
            self.q = q
            
            self.b = 0.5*(self.r - self.q - sigma**2 / 6)

            self.d1 = (np.log(self.St/self.K) + (self.b + self.sigma**2/2)*self.T)/(self.sigma*np.sqrt(self.T))
            self.d2 = self.d1 - self.sigma * np.sqrt(self.T)
 
            return True
        else:
            print("Error: Method not implemented")
            return False
    
    def price_using_monte_carlo(self, St, r, df, AveragePeriod, AverageType):
        return self._price_asian(St, r, df, AverageType, AveragePeriod)

In [20]:
class AsianCall(AsianOption):
    def __init__(self, T, PayoutType, K=0):
        super().__init__(T, PayoutType, K)
        self.Type = "Call"
    
    def price_closed_form(self, St, r, sigma, q=0):
        if super().price_closed_form(St, r, sigma, q): 
            return self.St * np.exp((self.b-self.r)*self.T) * norm.cdf(self.d1, 0, 1) - self.K * np.exp(-self.r*self.T) * norm.cdf(self.d2, 0, 1)

In [21]:
class AsianPut(AsianOption):
    def __init__(self, T, PayoutType, K=0):
        super().__init__(T, PayoutType, K)
        self.Type = "Put"
    
    def price_closed_form(self, St, r, sigma, q=0):
        if super().price_closed_form(St, r, sigma, q):
            return self.K * np.exp(-self.r*self.T) * norm.cdf(-self.d2, 0, 1) - self.St * np.exp((self.b-self.r)*self.T) * norm.cdf(-self.d1, 0, 1)

In [22]:
class ChooserOption(EuropeanCall, EuropeanPut):
    def __init__(self, T1, T2, K):
        self.T1 = T1
        self.T2 = T2
        self.K = K
    
    def price_closed_form(self, St, r, sigma, q=0):
        self.St = St
        self.r = r
        self.sigma = sigma
        self.q = q
        
        num_c = 1
        num_p = np.exp(-q*(self.T2-self.T1))
        self.K_chooser = self.K
        
        self.T = self.T1
        self.K = self.K_chooser*np.exp(-(self.r-self.q)*(self.T2-self.T1))
        p = EuropeanPut.price_using_BS(self, St, r, sigma, q)
        
        self.T = self.T2 
        self.K = self.K_chooser
        c = EuropeanCall.price_using_BS(self, St, r, sigma, q)
        
        return num_c * c + num_p * p
    
    def price_using_monte_carlo(self, St, r, df):
        self.T = self.T2 
        return self._price_chooser(St, r, df)

In [23]:
class FloatingStrikeLookbackOption(Option):
    def __init__(self, T):
        self.T = T
    
    def price_closed_form(self, St, r, sigma, q):
        self.St = St
        self.r = r
        self.sigma = sigma
        self.q = q
        
    def price_using_monte_carlo(self, St, r, df):
        return self._price_floating_strike_lookback(St, r, df)      

In [24]:
class FloatingStrikeLookbackCall(FloatingStrikeLookbackOption):
    def __init__(self, T):
        super().__init__(T)
        self.Type = "Call"
        
    def price_closed_form(self, St, r, sigma, q=0, Smin=0):
        super().price_closed_form(St, r, sigma, q)
        if Smin != 0:
            self.Smin = Smin
        else:
            self.Smin = self.St
        
        a1 = (np.log(self.St/self.Smin) + (self.r - self.q + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        a2 = a1 - self.sigma * np.sqrt(self.T)
        a3 = (np.log(self.St/self.Smin) + (-self.r + self.q + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        Y1 = (2 * np.log(self.St/self.Smin) * (self.r - self.q - 0.5 * sigma**2)) / (self.sigma**2)
        
        x1 = self.sigma**2 / (2*(self.r-self.q))
        x2 = norm.cdf(a2, 0, 1) - x1 * np.exp(Y1) * norm.cdf(-a3, 0, 1)
        
        return self.St * np.exp(-self.q * self.T) * norm.cdf(a1, 0, 1) - self.St * np.exp(-self.q * self.T) * x1 * norm.cdf(-a1, 0, 1) - self.Smin * np.exp(-self.r * self.T) * x2

In [25]:
class FloatingStrikeLookbackPut(FloatingStrikeLookbackOption):
    def __init__(self, T):
        super().__init__(T)
        self.Type = "Put"
        
    def price_closed_form(self, St, r, sigma, q=0, Smax=0):
        super().price_closed_form(St, r, sigma, q)
        if Smax != 0:
            self.Smax = Smax
        else:
            self.Smax = self.St
        
        b1 = (np.log(self.Smax/self.St) + (-self.r + self.q + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        b2 = b1 - self.sigma * np.sqrt(self.T)
        b3 = (np.log(self.Smax/self.St) + (self.r - self.q - 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        Y2 = (2 * np.log(self.Smax/self.St) * (self.r - self.q - 0.5 * sigma**2)) / (self.sigma**2)
        
        z1 = self.sigma**2 / (2*(self.r-self.q))
        z2 = norm.cdf(b1, 0, 1) - z1 * np.exp(Y2) * norm.cdf(-b3, 0, 1)
        
        return self.Smax * np.exp(-self.r * self.T) * z2 + self.St * np.exp(-self.q * self.T) * z1 * norm.cdf(-b2, 0, 1) - self.St * np.exp(-self.q * self.T) * norm.cdf(b2, 0, 1)

## Bewertung mehrer Optionen im Vergleich

In [52]:
#[Month, VDAX Close, DAX Close, ATM Strike]
options = [["FEB",23.50,13848.35,13850],
        ["MAR",21.89,14109.48,14100],
        ["APR",20.19,14569.39,14550],
        ["MAY",18.74,15368.39,15350],
        ["JUN",20.76,15416.64,15400],
        ["JUL",17.86,15673.64,15650],
        ["AUG",24.11,15133.20,15150],
        ["SEP",18.02,15925.73,15950],
        ["OCT",21.31,15701.42,15700],
        ["NOV",16.72,15474.47,15450],
        ["DEC",15.66,16148.64,16150]]

In [53]:
class Compare():
    def __init__(self, options, precision=2):
        self.options = options
        self.precision = precision
        
    def define_compare(self, option):
        pass
    
    def compare(self):

        for option in self.options:
            if option == self.options[0]:
                print(f"<{type(self).__name__}>\n")
            self.define_compare(option)
            
            self.c_difference = (self.call_price_mc - self.call_price_ref) / self.call_price_ref
            self.p_difference = (self.put_price_mc - self.put_price_ref) / self.put_price_ref

            print(f"{option[0]} C ==> Reference Price: {round(self.call_price_ref,self.precision)} MC Price: {round(self.call_price_mc,self.precision)} | Difference: {round(self.c_difference*100,self.precision)} %")
            print(f"{option[0]} P ==> Reference Price: {round(self.put_price_ref,self.precision)} MC Price: {round(self.put_price_mc,self.precision)} | Difference: {round(self.p_difference*100,self.precision)} %\n")

<b>European</b>

In [56]:
class EuropeanOptionsCompare(Compare):
    def define_compare(self, option):
            self.call = EuropeanCall(T=T, K=option[3])
            self.call_price_ref = self.call.price_using_BS(St=option[2], r=r, sigma=option[1]/100)
            self.call_price_mc = self.call.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)

            self.put = EuropeanPut(T=T, K=option[3])
            self.put_price_ref = self.put.price_using_BS(St=option[2], r=r, sigma=option[1]/100)
            self.put_price_mc = self.put.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)

In [57]:
european_options_compare = EuropeanOptionsCompare(options)
european_options_compare.compare()

<EuropeanOptionsCompare>

FEB C ==> Reference Price: 407.85 MC Price: 360.61 | Difference: -11.58 %
FEB P ==> Reference Price: 396.21 MC Price: 257.04 | Difference: -35.13 %

MAR C ==> Reference Price: 393.0 MC Price: 373.61 | Difference: -4.93 %
MAR P ==> Reference Price: 369.99 MC Price: 256.93 | Difference: -30.56 %

APR C ==> Reference Price: 380.04 MC Price: 391.17 | Difference: 2.93 %
APR P ==> Reference Price: 346.69 MC Price: 261.09 | Difference: -24.69 %

MAY C ==> Reference Price: 372.37 MC Price: 411.46 | Difference: 10.5 %
MAY P ==> Reference Price: 339.25 MC Price: 276.31 | Difference: -18.55 %

JUN C ==> Reference Price: 411.04 MC Price: 411.74 | Difference: 0.17 %
JUN P ==> Reference Price: 379.62 MC Price: 277.97 | Difference: -26.78 %

JUL C ==> Reference Price: 365.22 MC Price: 422.38 | Difference: 15.65 %
JUL P ==> Reference Price: 326.56 MC Price: 279.67 | Difference: -14.36 %

AUG C ==> Reference Price: 449.8 MC Price: 385.83 | Difference: -14.22 %
AUG P ==> Refere

<b>Digitals</b>

In [81]:
class CashOrNothingOptionsCompare(Compare):
    def define_compare(self, option):
            Q = 100
            
            self.call = DigitalCall(T=T, K=option[3], PayoutType="Cash")
            self.call_price_ref = self.call.price_closed_form(St=option[2], r=r, sigma=option[1]/100, Q=Q)
            self.call_price_mc = self.call.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN, Q=Q)

            self.put = DigitalPut(T=T, K=option[3], PayoutType="Cash")
            self.put_price_ref = self.put.price_closed_form(St=option[2], r=r, sigma=option[1]/100, Q=Q)
            self.put_price_mc = self.put.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN, Q=Q)

In [82]:
cash_or_nothing_options_compare = CashOrNothingOptionsCompare(options)
cash_or_nothing_options_compare.compare()

<CashOrNothingOptionsCompare>

FEB C ==> Reference Price: 48.96 MC Price: 55.27 | Difference: 12.88 %
FEB P ==> Reference Price: 50.94 MC Price: 44.64 | Difference: -12.38 %

MAR C ==> Reference Price: 49.56 MC Price: 55.81 | Difference: 12.61 %
MAR P ==> Reference Price: 50.34 MC Price: 44.1 | Difference: -12.41 %

APR C ==> Reference Price: 50.17 MC Price: 56.26 | Difference: 12.14 %
APR P ==> Reference Price: 49.74 MC Price: 43.65 | Difference: -12.24 %

MAY C ==> Reference Price: 50.28 MC Price: 56.17 | Difference: 11.73 %
MAY P ==> Reference Price: 49.63 MC Price: 43.73 | Difference: -11.88 %

JUN C ==> Reference Price: 49.93 MC Price: 56.09 | Difference: 12.34 %
JUN P ==> Reference Price: 49.97 MC Price: 43.81 | Difference: -12.33 %

JUL C ==> Reference Price: 50.63 MC Price: 56.38 | Difference: 11.36 %
JUL P ==> Reference Price: 49.28 MC Price: 43.53 | Difference: -11.67 %

AUG C ==> Reference Price: 48.38 MC Price: 54.6 | Difference: 12.85 %
AUG P ==> Reference Price: 51.52 MC 

In [79]:
class AssetOrNothingOptionsCompare(Compare):
    def define_compare(self, option):
            self.call = DigitalCall(T=T, K=option[3], PayoutType="Asset")
            self.call_price_ref = self.call.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.call_price_mc = self.call.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)
            
            self.put = DigitalPut(T=T, K=option[3], PayoutType="Asset")
            self.put_price_ref = self.put.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.put_price_mc = self.put.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)

In [80]:
asset_or_nothing_options_compare = AssetOrNothingOptionsCompare(options)
asset_or_nothing_options_compare.compare()

<AssetOrNothingOptionsCompare>

FEB C ==> Reference Price: 7189.01 MC Price: 8015.22 | Difference: 11.49 %
FEB P ==> Reference Price: 6659.34 MC Price: 5925.07 | Difference: -11.03 %

MAR C ==> Reference Price: 7380.9 MC Price: 8242.59 | Difference: 11.67 %
MAR P ==> Reference Price: 6728.58 MC Price: 5960.56 | Difference: -11.41 %

APR C ==> Reference Price: 7679.13 MC Price: 8576.41 | Difference: 11.68 %
APR P ==> Reference Price: 6890.26 MC Price: 6089.7 | Difference: -11.62 %

MAY C ==> Reference Price: 8089.69 MC Price: 9034.18 | Difference: 11.68 %
MAY P ==> Reference Price: 7278.7 MC Price: 6436.24 | Difference: -11.57 %

JUN C ==> Reference Price: 8100.91 MC Price: 9050.23 | Difference: 11.72 %
JUN P ==> Reference Price: 7315.73 MC Price: 6468.76 | Difference: -11.58 %

JUL C ==> Reference Price: 8288.47 MC Price: 9245.36 | Difference: 11.54 %
JUL P ==> Reference Price: 7385.17 MC Price: 6532.33 | Difference: -11.55 %

AUG C ==> Reference Price: 7779.97 MC Price: 8657.97 | Diff

<b>Asians</b>

In [83]:
class AveragePriceOptionsCompare(Compare):
    def define_compare(self, option):
            self.call = AsianCall(T=T, K=option[3], PayoutType="AveragePrice")
            self.call_price_ref = self.call.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.call_price_mc = self.call.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN, AveragePeriod=24, AverageType="Geometric")
            
            self.put = AsianPut(T=T, K=option[3], PayoutType="AveragePrice")
            self.put_price_ref = self.put.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.put_price_mc = self.put.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN, AveragePeriod=24, AverageType="Geometric")

In [84]:
average_price_options_compare = AveragePriceOptionsCompare(options)
average_price_options_compare.compare()

<AveragePriceOptionsCompare>

FEB C ==> Reference Price: 231.47 MC Price: 211.76 | Difference: -8.52 %
FEB P ==> Reference Price: 232.59 MC Price: 164.19 | Difference: -29.41 %

MAR C ==> Reference Price: 225.59 MC Price: 221.93 | Difference: -1.62 %
MAR P ==> Reference Price: 214.75 MC Price: 162.32 | Difference: -24.42 %

APR C ==> Reference Price: 220.57 MC Price: 234.57 | Difference: 6.35 %
APR P ==> Reference Price: 198.96 MC Price: 163.42 | Difference: -17.86 %

MAY C ==> Reference Price: 216.12 MC Price: 246.26 | Difference: 13.95 %
MAY P ==> Reference Price: 194.69 MC Price: 173.27 | Difference: -11.0 %

JUN C ==> Reference Price: 237.54 MC Price: 246.02 | Difference: 3.57 %
JUN P ==> Reference Price: 218.83 MC Price: 174.6 | Difference: -20.21 %

JUL C ==> Reference Price: 213.31 MC Price: 253.92 | Difference: 19.04 %
JUL P ==> Reference Price: 186.17 MC Price: 174.6 | Difference: -6.21 %

AUG C ==> Reference Price: 252.06 MC Price: 223.25 | Difference: -11.43 %
AUG P ==> Refe

<b>Chooser</b>

In [89]:
class ChooserOptionsCompare(Compare):
    def define_compare(self, option):
            self.option = ChooserOption(T1=T/2, T2=T, K=option[3])
            self.price_ref = self.option.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.price_mc = self.option.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)
    def compare(self):
        for option in self.options:
            if option == self.options[0]:
                print(f"<{type(self).__name__}>\n")
            self.define_compare(option)

            self.difference = (self.price_mc - self.price_ref) / self.price_ref

            print(f"{option[0]} ==> Reference Price: {round(self.price_ref,self.precision)} MC Price: {round(self.price_mc,self.precision)} | Difference: {round(self.difference*100,self.precision)} %\n")

In [90]:
chooser_options_compare = ChooserOptionsCompare(options)
chooser_options_compare.compare()

<ChooserOptionsCompare>

FEB ==> Reference Price: 686.36 MC Price: 523.54 | Difference: -23.72 %

MAR ==> Reference Price: 651.36 MC Price: 535.02 | Difference: -17.86 %

APR ==> Reference Price: 620.5 MC Price: 554.08 | Difference: -10.7 %

MAY ==> Reference Price: 607.59 MC Price: 584.13 | Difference: -3.86 %

JUN ==> Reference Price: 675.04 MC Price: 585.65 | Difference: -13.24 %

JUL ==> Reference Price: 590.73 MC Price: 596.71 | Difference: 1.01 %

AUG ==> Reference Price: 769.82 MC Price: 570.54 | Difference: -25.89 %

SEP ==> Reference Price: 605.72 MC Price: 599.58 | Difference: -1.01 %

OCT ==> Reference Price: 705.69 MC Price: 594.17 | Difference: -15.8 %

NOV ==> Reference Price: 546.14 MC Price: 589.32 | Difference: 7.9 %

DEC ==> Reference Price: 533.47 MC Price: 610.6 | Difference: 14.46 %



<b>Lookbacks</b>

In [91]:
class FloatingStrikeLookbackOptionsCompare(Compare):
    def define_compare(self, option):
            self.call = FloatingStrikeLookbackCall(T=T)
            self.call_price_ref = self.call.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.call_price_mc = self.call.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)
            
            self.put = FloatingStrikeLookbackPut(T=T)
            self.put_price_ref = self.put.price_closed_form(St=option[2], r=r, sigma=option[1]/100)
            self.put_price_mc = self.put.price_using_monte_carlo(St=option[2], r=r, df=DataTimeGAN)

In [92]:
floating_strike_lookback_options_compare = FloatingStrikeLookbackOptionsCompare(options)
floating_strike_lookback_options_compare.compare()

<FloatingStrikeLookbackOptionsCompare>

FEB C ==> Reference Price: 792.64 MC Price: 634.48 | Difference: -19.95 %
FEB P ==> Reference Price: 816.05 MC Price: 565.13 | Difference: -30.75 %

MAR C ==> Reference Price: 753.9 MC Price: 646.44 | Difference: -14.25 %
MAR P ==> Reference Price: 772.8 MC Price: 575.79 | Difference: -25.49 %

APR C ==> Reference Price: 719.74 MC Price: 667.52 | Difference: -7.26 %
APR P ==> Reference Price: 734.25 MC Price: 594.56 | Difference: -19.03 %

MAY C ==> Reference Price: 706.21 MC Price: 704.12 | Difference: -0.29 %
MAY P ==> Reference Price: 717.35 MC Price: 627.16 | Difference: -12.57 %

JUN C ==> Reference Price: 782.46 MC Price: 706.33 | Difference: -9.73 %
JUN P ==> Reference Price: 799.54 MC Price: 629.13 | Difference: -21.31 %

JUL C ==> Reference Price: 687.35 MC Price: 718.11 | Difference: 4.47 %
JUL P ==> Reference Price: 696.3 MC Price: 639.62 | Difference: -8.14 %

AUG C ==> Reference Price: 887.95 MC Price: 693.35 | Difference: -21.92 %
A